In [59]:
# https://www.tensorflow.org/datasets/keras_example
# without transfer learning

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import logging, os
import pandas as pd

logging.basicConfig(level=logging.INFO)
# logging.basicConfig(level=logging.DEBUG)

BASE_PATH = "../../../../../local_data/practice/tfds/"
DATA_PATH = "../../../../../local_data/tfds/"
OUTPUT_PATH = BASE_PATH+"mnist_simple_fit/"
os.system("mkdir -p " + OUTPUT_PATH)


(train_ds, test_ds), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
print("train_ds:",train_ds)
print(f"Number of training samples: {train_ds.cardinality()}")
print(f"Number of test samples: {test_ds.cardinality()}")

# plt.figure(figsize=(10, 10))
# for i, (image, label) in enumerate(train_ds.take(9)):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(image)
#     plt.title(int(label))
#     plt.axis("off")

INFO:absl:Load dataset info from /home/tmeng12/tensorflow_datasets/mnist/3.0.1
INFO:absl:Fields info.[citation, splits, supervised_keys, module_name] from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset mnist (/home/tmeng12/tensorflow_datasets/mnist/3.0.1)
INFO:absl:Constructing tf.data.Dataset mnist for split ['train', 'test'], from /home/tmeng12/tensorflow_datasets/mnist/3.0.1


train_ds: <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
Number of training samples: 60000
Number of test samples: 10000


In [ ]:
c1=tfds.as_dataframe(train_ds.take(20), ds_info)
col1 = c1['label']
# col1 = pd.DataFrame(train_ds, columns=["image","train_ds"])
print(col1)

In [ ]:
batch_size=16

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

train_ds = train_ds.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.cache()
# train_ds = train_ds.shuffle(ds_info.splits['train'].num_examples)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
test_ds = test_ds.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size)
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(batch_size, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.summary()
model.fit(
    train_ds,
    epochs=1,
    validation_data=test_ds,
)

In [ ]:
(train_ds, test_ds), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
print(f"Number of training samples: {train_ds.cardinality()}")
print(f"Number of test samples: {test_ds.cardinality()}")


In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

train_ds = train_ds.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.cache()
# train_ds = train_ds.shuffle(ds_info.splits['train'].num_examples)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
c1=tfds.as_dataframe(train_ds.take(5000), ds_info)
col1 = c1['label']
print(col1.shape)

pred = model.predict(train_ds)
# print("pred\n",pred)
col2 = pd.DataFrame(pred, columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
# print(col2)

compare = pd.concat([col1, col2], axis=1)
print(compare)
compare.to_csv(OUTPUT_PATH + "pred_train.csv", index=False)

In [ ]:
pred = model.predict(test_ds)
# print("pred\n",pred)
# col2 = pd.DataFrame(pred[0:20], columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
col2 = pd.DataFrame(pred, columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
# print(col2)

compare = pd.concat([col1, col2], axis=1)
print(compare)
compare.to_csv(OUTPUT_PATH + "pred_test.csv", index=False)